# Bathy illustrations
This notebook plots the MEDWEST60 bathymetry

In [2]:
## standart libraries
import os,sys
import numpy as np

from scipy.signal import argrelmax
from scipy.stats import linregress

# xarray
import xarray as xr

# plot
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean

# custom tools
import lib_medwest60 as slx

%matplotlib inline 

In [4]:
diro='/mnt/meom/workdir/lerouste/MEDWEST60/DIAGS/'

* reads bathy

In [5]:
machine='CAL1'
nav_lat,nav_lon,bathy = slx.readonlybathy(machine)

* also reads sst and mask

In [7]:

varna="sosstsst"
typ="gridT-2D"
naexp="NO"
CONFIGCASE='MEDWEST60-GSL14'
ens='ens01'
machine='CAL1'
nav_lat_ref,nav_lon_ref,mask,sst,varname,latexvarname=slx.readallmbs(machine=machine,NMBtot=2,typ=typ,varna=varna,CONFIGCASEmed=CONFIGCASE,ens=ens)
sst.attrs['experiment_short_name'] =  naexp


/mnt/meom/workdir/lerouste/MEDWEST60/MEDWEST60-GSL14-S/ens01/1h/gridT-2D/
001MEDWEST60-GSL14-ens01_1h_*gridT-2D_*.nc
/mnt/meom/workdir/lerouste/MEDWEST60/MEDWEST60-GSL14-S/ens01/1h/gridT-2D/
002MEDWEST60-GSL14-ens01_1h_*gridT-2D_*.nc


* plot bathy

In [ ]:
ie=1
it=0
Lregion=[454,804,176,526]
region=[470,720,95,415]
#region4=[470,720,115,365]
gstyle='darkstyle'
plto = "bathyplot"    


#====================================
# data to plot 

data2plot = (bathy*1e-3).where(mask.isel(z=0)>0).squeeze().to_masked_array()

namo = plto+'_'+str(it)+'.png'


#========= Plot settings ===============

levbounds=[0.,3.250,0.050]

# customed colormap
cmap,norm = slx.mycolormap(levbounds,cm_base=cmocean.cm.ice_r,cu='w',co='k')

#========= PLOT ===============
fig3,(ax) = plt.subplots(1, 1, figsize=[16, 12],facecolor='w')

# main plot
cs,ax = slx.plotmapMEDWEST_gp(fig3,ax,data2plot,cmap,norm,plto=plto,gridpts=True,gridptsgrid=True,gstyle=gstyle)

lalpha=0.2
lwidthgrid=1.
gridinc=200
lstylegrid=(0, (5, 5))
if (gstyle=='lightstyle'):
        cmap.set_bad('w')
        lcolorgrid="#585858" # "#D8D8D8"
        tcolorgrid='#848484'#"#848484"         
        
if (gstyle=='darkstyle'):
        lcolorgrid="w" # "#D8D8D8"
        tcolorgrid='#848484'#"#848484"  

#ax = plt.gca()
ax.xaxis.set_major_locator(mticker.MultipleLocator(gridinc))
ax.yaxis.set_major_locator(mticker.MultipleLocator(gridinc))   
ax.tick_params(axis='x', colors=tcolorgrid)
ax.tick_params(axis='y', colors=tcolorgrid)
ax.grid(which='major',linestyle=lstylegrid,color=lcolorgrid,alpha=lalpha,linewidth=lwidthgrid)
ax.axhline(y=1.,xmin=0, xmax=883,zorder=10,color=lcolorgrid,linewidth=lwidthgrid,linestyle=lstylegrid,alpha=lalpha )

# add date
tcolordate="848484"
tsizedate=14

plt.axhline(y=350.,xmin=200, xmax=400,zorder=10,color='r',linewidth=3,linestyle='-',alpha=1 )


# add colorbar
cbon=True
if cbon:
    tlabel=latexvarname+slx.textunit(varname)
    tlabel="bathymetry (km)"
    cb = slx.addcolorbar(fig3,cs,ax,levbounds,levincr=10,tformat="%.1f",tlabel=tlabel,facmul=1,orientation='horizontal',tc='w')


showzoom=False
if showzoom:
    #region=region4
    xbox=[region[0],region[0],region[1],region[1],region[0]]
    ybox=[region[2],region[3],region[3],region[2],region[2]]
    ax.plot(xbox, ybox, color="#FFFF00", alpha=1,linewidth=3, linestyle="-",solid_capstyle='round', zorder=2)
    #ax.annotate("zoom 0",xy=(460,485),xycoords='data', color="#0B3861",size=12)

showzoom2=False
if showzoom2:
    #region=region4
    xbox=[Lregion[0],Lregion[0],Lregion[1],Lregion[1],Lregion[0]]
    ybox=[Lregion[2],Lregion[3],Lregion[3],Lregion[2],Lregion[2]]
    ax.plot(xbox, ybox, color="#FFFF00", alpha=1,linewidth=3, linestyle=":",solid_capstyle='round', zorder=2)

plt.show()

# Save fig in png, resolution dpi    
slx.saveplt(fig3,diro,namo,dpifig=300)
    